# SQLite Murder Mystery 
Brandon Stibich

Intro: 

The database for this exercise was found on Kaggle, however it is originally sourced from here https://mystery.knightlab.com. 

The exercise presents a database schema full of information, and a tiny lead in which to solve a murder. I am given the starting point of locating the crime scene report for a *murder* that occurred on *Jan.15, 2018* in *SQL City*. 

These next query is suggested as the starting point from the website.

The queries are executed in SQLite Studio, this document is just to save the queries as well as the thought process behind them for documentation purposes.

 

In [ ]:
SELECT name 
  FROM sqlite_master
 where type = 'table'

RESULTS

crime_scene_report

drivers_license

person

facebook_event_checkin

interview

get_fit_now_member

get_fit_now_check_in

income

solution

===================================================================================================================

I see the table 'crime_scene_report' and am certainly drawn to start there to find the report I need.

However I'd also like to note that the check in tables are interesting and I'm curious to find how those play into this puzzle.

In [ ]:
SELECT description
  FROM crime_scene_report
 WHERE date == 20180115 AND 
       type == "murder" AND 
       city == "SQL City";

RESULTS

"Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave"."

===================================================================================================================

There was only 1 crime that matches the description given. It seems we have 2 witnesses that we can get info from. I think I'll start by trying to find their names via the person table, as it has a column for address street name. Then I will see if we have an interview with the witnesses.

In [ ]:
SELECT *
  FROM person
 WHERE address_street_name == "Franklin Ave" AND 
       name LIKE "Annabel%";


RESULTS

16371	Annabel Miller	490173	103	Franklin Ave	318771143

===================================================================================================================



1 down lets find the other witness

In [ ]:
SELECT *
  FROM person
 WHERE address_street_name == "Northwestern Dr"
 ORDER BY address_number DESC;


RESULTS

*14887	Morty Schapiro	118009	4919	Northwestern Dr	111564949*

17729	Lasonya Wildey	439686	3824	Northwestern Dr	917817122

....

===================================================================================================================


Looks like Mr. Morty Schapiro is our witness here, he's the last house on the street.

Lets see if we have interviews with these witnesses

In [ ]:
--Schapiro
SELECT *
  FROM interview
 WHERE person_id == 14887;

--Miller
SELECT *
  FROM interview
 WHERE person_id == 16371;


RESULTS

SCHAPIRO:
"I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W"."

MILLER:
"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."

===================================================================================================================


Now things are getting juicy! A few key notes from these interviews: The suspect is a member of "Get Fit Now" with at least a gold member status and a membership number including "48Z". I can get all that info from the get_fit_now_member table. I can then cross reference that with the get_fit_now_check_in table to check if the person was also there on jan 9th. After I find a suspect I can check the drivers_lisence table to confirm the car. 

Lets see if things go so simply.


In [ ]:
SELECT *
  FROM get_fit_now_member
 WHERE membership_status == "gold" AND 
       id LIKE "%48Z%";

RESULTS

48Z7A	28819	Joe Germuska	20160305	gold

48Z55	67318	Jeremy Bowers	20160101	gold

===================================================================================================================

Looks like we got more than one matching member, lets try and narrow it down based on who was at the gym on the 9th.

In [ ]:
SELECT *
  FROM get_fit_now_check_in
 WHERE check_in_date == 20180109 AND 
       membership_id LIKE "%48Z%";


RESULTS

48Z7A	20180109	1600	1730

48Z55	20180109	1530	1700

===================================================================================================================

Darn they were both there on the 9th. Hopefully the plate can get us the info we need.

In [ ]:
SELECT *
  FROM drivers_license
 WHERE plate_number LIKE "%H42W%";


RESULTS

183779	21	65	blue	blonde	female	H42W0X	Toyota	Prius

423327	30	70	brown	brown	male	0H42W2	Chevrolet	Spark LS

664760	21	71	black	black	male	4H42WR	Nissan	Altima

===================================================================================================================

Theres a few with "H42W" in their plates, lets check with the person table to see if either of our gym members are owners of one of these cars.

In [ ]:
SELECT *
  FROM person
 WHERE license_id == 183779 OR 
       license_id == 423327 OR 
       license_id == 664760;


RESULTS

51739	Tushar Chandra	664760	312	Phi St	137882671

67318	Jeremy Bowers	423327	530	Washington Pl, Apt 3A	871539279

78193	Maxine Whitely	183779	110	Fisk Rd	137882671

===================================================================================================================

I believe we have our guy with this. Mr. Jeremy Bowers is a gold member of Get Fit Now Gym, with a membership id containing '48Z'. He was at said gym on both the date of the crime Jan.18th and Jan.9th. And he owns a Chevy Spark which has the 'H4W' plate that was seen by Mr. Schapiro.

Lets see if the solution is correct!

In [ ]:
--this query template was given on the website

INSERT INTO solution VALUES (1, 'Jeremy Bowers');
        
        SELECT value FROM solution;

RESULTS

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

===================================================================================================================

I did it!! That was a lot of fun and I am absolutely going for this challenge they talk about!

In [ ]:
SELECT *
  FROM interview
 WHERE person_id == 67318;


RESULTS

67318	"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.
"

===================================================================================================================

I don't suspect this should be tough to find with all these details, lets start with the drivers_license table since it has car, height, and hair info.



In [ ]:
SELECT *
  FROM drivers_license
 WHERE hair_color == "red" AND 
       (64 < height < 68) AND 
       car_make == "Tesla" AND 
       car_model == "Model S";


RESULTS

202298	68	66	green	red	female	500123	Tesla	Model S

291182	65	66	blue	red	female	08CM64	Tesla	Model S

736081	79	69	brown	red	male	GCAQ6Y	Tesla	Model S

918773	48	65	black	red	female	917UU3	Tesla	Model S

===================================================================================================================

Alright we got a few hits on this one, lets see who among them attended that Symphony 3 times in 2017 though.

First we have to transfer the license_id to a person_id

In [ ]:
SELECT *
  FROM person
 WHERE license_id == 202298 OR 
       license_id == 291182 OR 
       license_id == 736081 OR 
       license_id == 918773;


RESULTS

57410	Cletus Zoeller	736081	2987	Kingham Way	924648898

78881	Red Korb	918773	107	Camerata Dr	961388910

90700	Regina George	291182	332	Maple Ave	337169072

99716	Miranda Priestly	202298	1883	Golden Ave	987756388

===================================================================================================================

Quickly plug those person_ids into the facebook check ins.


In [ ]:
SELECT *
  FROM facebook_event_checkin
 WHERE event_name == "SQL Symphony Concert" AND 
       date LIKE "201712%" AND 
       person_id == 57410 OR 
       person_id == 78881 OR 
       person_id == 90700 OR 
       person_id == 99716;


RESULTS

99716	1143	SQL Symphony Concert	20171206

99716	1143	SQL Symphony Concert	20171212

99716	1143	SQL Symphony Concert	20171229


===================================================================================================================

And it looks like person 99716 or Ms. Miranda Priestly is our woman. Lets confirm with a solution check.

In [ ]:
INSERT INTO solution VALUES (1, 'Miranda Priestly');
        
        SELECT value FROM solution;

RESULTS

"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"

===================================================================================================================

Yeehaw! Man that was a lot of fun!